In [1]:
!pip install -U xgboost -f /kaggle/input/xgboost-python-package/ --no-index

Looking in links: /kaggle/input/xgboost-python-package/
^C
ERROR: Operation cancelled by user


In [2]:
import numpy as np
import pandas as pd
import json

import seaborn as sns
import matplotlib.pyplot as plt
from colorama import Fore, Style, init;
from sklearn.metrics import mean_squared_error

import xgboost as xgb
import lightgbm as lgb
import torch

#Geolocation
from geopy.geocoders import Nominatim

pd.set_option('display.max_columns', 100)
DEBUG=False

KeyboardInterrupt: 

In [ ]:
# using GPU or CPU
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

In [ ]:
# just some coloring and printing functions
# first one prints the size and first row of the dataset
# second one is just coloring
def display_df(df, name):
    '''Display df shape and first row '''
    PrintColor(text = f'{name} data has {df.shape[0]} rows and {df.shape[1]} columns. \n ===> First row:')
    display(df.head(1))

def PrintColor(text:str, color = Fore.BLUE, style = Style.BRIGHT):
    '''Prints color outputs using colorama of a text string'''
    print(style + color + text + Style.RESET_ALL); 

In [ ]:
# just reading all datasets
train = pd.read_csv("/kaggle/input/predict-energy-behavior-of-prosumers/train.csv")
client = pd.read_csv("/kaggle/input/predict-energy-behavior-of-prosumers/client.csv")
historical_weather = pd.read_csv("/kaggle/input/predict-energy-behavior-of-prosumers/historical_weather.csv")
forecast_weather = pd.read_csv("/kaggle/input/predict-energy-behavior-of-prosumers/forecast_weather.csv")
electricity = pd.read_csv("/kaggle/input/predict-energy-behavior-of-prosumers/electricity_prices.csv")
gas = pd.read_csv("/kaggle/input/predict-energy-behavior-of-prosumers/gas_prices.csv")
location = pd.read_csv("/kaggle/input/fabiendaniels-mapping-locations-and-county-codes/county_lon_lats.csv").drop(columns=["Unnamed: 0"])

In [ ]:
# using the coloring and printing functions to show datasets
display_df(train, "train")
display_df(client, "client")
display_df(historical_weather, "historical_weather")
display_df(forecast_weather, "forecast_weather")
display_df(electricity, "electricity")
display_df(gas, "gas")
display_df(location, "location")

In [ ]:
# open the county id to name file (json format)
with open("/kaggle/input/predict-energy-behavior-of-prosumers/county_id_to_name_map.json") as f:
    county_codes = json.load(f)
pd.DataFrame(county_codes, index=[0])

In [ ]:
class FeatureProcessorClass():
    def __init__(self):         
        # Take columns from different datasets 
        # and combine into one later: weather, client, electricity, gas
        self.weather_join = ['datetime', 'county', 'data_block_id']
        self.gas_join = ['data_block_id']
        self.electricity_join = ['datetime', 'data_block_id']
        self.client_join = ['county', 'is_business', 'product_type', 'data_block_id']
        
        # Columns of latitude & longitude
        self.lat_lon_columns = ['latitude', 'longitude']
        
        # Aggregate stats 
        self.agg_stats = ['mean'] #, 'min', 'max', 'std', 'median']
        
        # Categorical columns (specify for XGBoost)
        self.category_columns = ['county', 'is_business', 'product_type', 'is_consumption', 'data_block_id']

    def create_new_column_names(self, df, suffix, columns_no_change):
        # Change column names by given suffix, check column name
        # if it is not in columns_no_change, change name into col + suffix
        # and return the data
        df.columns = [col + suffix 
                      if col not in columns_no_change
                      else col
                      for col in df.columns
                      ]
        return df 

    def flatten_multi_index_columns(self, df):
        # list A: (if it is not empty) column names
        # list B is the final list: for each column, "_".join(list A)
        df.columns = ['_'.join([col for col in multi_col if len(col)>0]) 
                      for multi_col in df.columns]
        return df
    
    def create_data_features(self, data):
        # format all the datetime
        
        # To datetime
        data['datetime'] = pd.to_datetime(data['datetime'])
        
        # Time period features
        data['date'] = data['datetime'].dt.normalize()
        data['year'] = data['datetime'].dt.year
        data['quarter'] = data['datetime'].dt.quarter
        data['month'] = data['datetime'].dt.month
        data['week'] = data['datetime'].dt.isocalendar().week
        data['hour'] = data['datetime'].dt.hour
        
        # Day features
        data['day_of_year'] = data['datetime'].dt.day_of_year
        data['day_of_month']  = data['datetime'].dt.day
        data['day_of_week'] = data['datetime'].dt.day_of_week
        return data

    def create_client_features(self, client):
        # Apply first two def to change some of the names in client list
        client = self.create_new_column_names(client, 
                                           suffix='_client',
                                           columns_no_change = self.client_join
                                          )       
        return client
    
    def create_historical_weather_features(self, historical_weather):
        # Formuate the datetime
        historical_weather['datetime'] = pd.to_datetime(historical_weather['datetime'])
        
        # Add column county into historical weather
        historical_weather[self.lat_lon_columns] = historical_weather[self.lat_lon_columns].astype(float).round(1)
        historical_weather = historical_weather.merge(location, how = 'left', on = self.lat_lon_columns)

        # Apply first two def to change some of the names in historical weather list
        # the column no change is historical weather + location
        historical_weather = self.create_new_column_names(historical_weather,
                                                          suffix='_h',
                                                          columns_no_change = self.lat_lon_columns + self.weather_join
                                                          ) 
        
        # for column in hist weather, if it is not in the combined list, append into agg_columns
        # make a dict that each col in agg_columns has the agg statistical info: see agg_stat in the first def
        # grouping according to the self.weather_join
        agg_columns = [col for col in historical_weather.columns if col not in self.lat_lon_columns + self.weather_join]
        agg_dict = {agg_col: self.agg_stats for agg_col in agg_columns}
        historical_weather = historical_weather.groupby(self.weather_join).agg(agg_dict).reset_index() 
        
        # Flatten the multi column aggregates
        historical_weather = self.flatten_multi_index_columns(historical_weather) 
        
        # Test set has 1 day offset for hour<11 and 2 day offset for hour>11
        historical_weather['hour_h'] = historical_weather['datetime'].dt.hour
        historical_weather['datetime'] = (historical_weather
                                               .apply(lambda x: 
                                                      x['datetime'] + pd.DateOffset(1) 
                                                      if x['hour_h']< 11 
                                                      else x['datetime'] + pd.DateOffset(2),
                                                      axis=1)
                                              )
        
        return historical_weather
    
    def create_forecast_weather_features(self, forecast_weather):
        # drop the origin_date and rename the columns
        forecast_weather = (forecast_weather
                            .rename(columns = {'forecast_datetime': 'datetime'})
                            .drop(columns = 'origin_datetime')
                           )
        
        # format the datetime
        forecast_weather['datetime'] = (pd.to_datetime(forecast_weather['datetime'])
                                        .dt
                                        .tz_localize(None)
                                       )

        # add county into forecast_weather dataset
        # add the location info into the dataset
        forecast_weather[self.lat_lon_columns] = forecast_weather[self.lat_lon_columns].astype(float).round(1)
        forecast_weather = forecast_weather.merge(location, how = 'left', on = self.lat_lon_columns)
        
        # Apply first two def to change some of the names in forecast weather and lat_lon
        forecast_weather = self.create_new_column_names(forecast_weather,
                                                        suffix='_f',
                                                        columns_no_change = self.lat_lon_columns + self.weather_join
                                                        ) 
        
        # for column in fore. weather, if it is not in the combined list, append into agg_columns
        # make a dict that each col in agg_columns has the agg statistical info: see agg_stat in the first def
        # grouping according to the self.weather_join
        agg_columns = [col for col in forecast_weather.columns if col not in self.lat_lon_columns + self.weather_join]
        agg_dict = {agg_col: self.agg_stats for agg_col in agg_columns}
        forecast_weather = forecast_weather.groupby(self.weather_join).agg(agg_dict).reset_index() 
        
        # Flatten the multi column aggregates
        forecast_weather = self.flatten_multi_index_columns(forecast_weather)     
        return forecast_weather

    def create_electricity_features(self, electricity):
        # Format datetime in electricity
        electricity['forecast_date'] = pd.to_datetime(electricity['forecast_date'])
        
        # Test set has 1 day offset
        electricity['datetime'] = electricity['forecast_date'] + pd.DateOffset(1)
        
        # Apply first two def to change names in electricity
        electricity = self.create_new_column_names(electricity, 
                                                   suffix='_electricity',
                                                   columns_no_change = self.electricity_join
                                                  )             
        return electricity

    def create_gas_features(self, gas):
        # Mean gas price
        gas['mean_price_per_mwh'] = (gas['lowest_price_per_mwh'] + gas['highest_price_per_mwh'])/2
        
        # Apply first two def to change names in gas
        gas = self.create_new_column_names(gas, 
                                           suffix='_gas',
                                           columns_no_change = self.gas_join
                                          )       
        return gas
    
    def __call__(self, data, client, historical_weather, forecast_weather, electricity, gas):
        # Create and prepare features for all dataset
        data = self.create_data_features(data)
        client = self.create_client_features(client)
        historical_weather = self.create_historical_weather_features(historical_weather)
        forecast_weather = self.create_forecast_weather_features(forecast_weather)
        electricity = self.create_electricity_features(electricity)
        gas = self.create_gas_features(gas)
        
        # Now add all dataset into one:
        df = data.merge(client, how='left', on = self.client_join)
        df = df.merge(historical_weather, how='left', on = self.weather_join)
        df = df.merge(forecast_weather, how='left', on = self.weather_join)
        df = df.merge(electricity, how='left', on = self.electricity_join)
        df = df.merge(gas, how='left', on = self.gas_join)
        
        # Change columns to categorical for XGBoost
        df[self.category_columns] = df[self.category_columns].astype('category')
        return df

In [ ]:
def create_revealed_targets_train(data, N_day_lags):
    # copy necessary info
    original_datetime = data['datetime']
    revealed_targets = data[['datetime', 'prediction_unit_id', 'is_consumption', 'target']].copy()
    
    # first calculate the 
    for day_lag in range(2, N_day_lags+1):
        revealed_targets['datetime'] = original_datetime + pd.DateOffset(day_lag)
        # now keep the original and using on ="List" as pivot
        # only add target_days_ago (original is left)
        data = data.merge(revealed_targets, 
                          how='left', 
                          on = ['datetime', 'prediction_unit_id', 'is_consumption'],
                          suffixes = ('', f'_{day_lag}_days_ago')
                         )
    return data

In [ ]:
%%time
# control the number of days to train (at least 2 days)
N_day_lags = 15

FeatureProcessor = FeatureProcessorClass()

data = FeatureProcessor(data = train.copy(),
                      client = client.copy(),
                      historical_weather = historical_weather.copy(),
                      forecast_weather = forecast_weather.copy(),
                      electricity = electricity.copy(),
                      gas = gas.copy(),
                     )

df = create_revealed_targets_train(data.copy(), 
                                  N_day_lags = N_day_lags)

In [ ]:
target = "target"
# cleaning, ditch null in target & reset index
df = df[df[target].notnull()].reset_index(drop=True)

train_block_id = list(range(0, 500))

# first 500 data_block_ids are used for training, if the 
# data_block_id IS IN the list, then it is train set, or else test set
tr = df[df["data_block_id"].isin(train_block_id)]

#others are for validation
val = df[~df["data_block_id"].isin(train_block_id)]

In [ ]:
no_features = ['date', 
                'latitude', 
                'longitude', 
                'data_block_id', 
                'row_id',
                'hours_ahead',
                'hour_h',
               ]

remove_columns = []
features = []

# prepare the feature we want to use, delete the unnecessary ones:
# gather the columns in df that is unnecessary for analysis
for col in df.columns:
    for no_feature in no_features:
        if no_feature in col:
            remove_columns.append(col)

remove_columns.append(target)

# extract the feature that is necessary
for col in df.columns:
    if col not in remove_columns:
        features.append(col)
        
PrintColor(f'There are {len(features)} features for training: {features}')

In [ ]:
tr['week']=tr['week'].astype(np.int32)
val['week']=val['week'].astype(np.int32)

# **1. XGBoost approach**

In [ ]:
%%time
# set up xgb
xgb_reg = xgb.XGBRegressor(device = device, enable_categorical=True, objective = "reg:absoluteerror", n_estimators = 2 if DEBUG else 1500)#, early_stopping_rounds=100)
# fitting
xgb_reg.fit(X = tr[features], y = tr[target], eval_set = [(tr[features], tr[target]), (val[features], val[target])], verbose=True)

In [ ]:
# print the best iteration that has smallest MAE
print('Early stopping on best iteration #', xgb_reg.best_iteration, 'with MAE error on validation set of', xgb_reg.best_score)

In [ ]:
# plotting both training and validation MAE
results = xgb_reg.evals_result_
train_mae, val_mae = results['validation_0']["mae"], results['validation_1']["mae"]
itern=list(range(len(train_mae)))

plt.plot(itern, train_mae, label="Training MAE")
plt.plot(itern, val_mae, label="Validation MAE")
plt.title("XGBRegressor: MAE vs Iteration number")
plt.xlabel("Iteration number")
plt.ylabel("MAE")
plt.legend()
plt.show()

In [ ]:
# plotting the top 20 important features
# starting from the most important one
data_imp = pd.DataFrame({"feature_name": xgb_reg.feature_names_in_, "importance": xgb_reg.feature_importances_})
data_imp

sorted_data_imp = data_imp.sort_values(by="importance", ascending=False)[:20].plot.barh(y="importance", x="feature_name", label = "importance")
sorted_data_imp.invert_yaxis()

# 2. LightBGM

In [ ]:
%%time
# set up lgb
gbm_reg = lgb.LGBMRegressor(objecive= "regression_l1", n_estimators = 2 if DEBUG else 1500)#, early_stopping_rounds = 100)
# fitting
gbm_reg.fit(tr[features], tr[target], eval_set = [(tr[features], tr[target]), (val[features], val[target])], eval_metric="mae")#, verbose=True)

In [ ]:
# print the best iteration that has smallest MAE
print('Early stopping on best iteration #', gbm_reg.best_iteration_, 'with MAE error on validation set of', gbm_reg.best_score_['valid_1']['l1'])

In [ ]:
# plotting both training and validation MAE
results2 = gbm_reg.evals_result_
train2_mae, val2_mae = results2['valid_0']["l1"], results2['valid_1']["l1"]
itern2=list(range(len(train2_mae)))

plt.plot(itern2, train2_mae, label="Training MAE")
plt.plot(itern2, val2_mae, label="Validation MAE")
plt.title("LGBMRegressor: MAE vs Iteration number")
plt.xlabel("Iteration number")
plt.ylabel("MAE")
plt.legend()
plt.show()

In [ ]:
# plotting the top 20 important features
# starting from the most important one
# normalized before showing
gbm_reg_imp=gbm_reg.feature_importances_/np.sum(gbm_reg.feature_importances_)
data_imp2 = pd.DataFrame({"feature_name": gbm_reg.feature_name_, "importance": gbm_reg_imp})
data_imp2

sorted_data_imp2 = data_imp2.sort_values(by="importance", ascending=False)[:20].plot.barh(y="importance", x="feature_name", label = "importance")
sorted_data_imp2.invert_yaxis()